In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pickle

Load Data

In [4]:
def get_grid_key(lat, lon):
    return (round(lat / GRID_RESOLUTION), round(lon / GRID_RESOLUTION))

def sample_line_count_estimate(tar_path, target_csv="observations", sample_bytes=100_000_000):
    """
    Estimate total line count by sampling the first few MB of the file inside the tar.gz.
    """
    with tarfile.open(tar_path, "r:gz") as tar:
        for member in tar:
            if member.name.endswith(".csv") and target_csv in member.name:
                file_obj = tar.extractfile(member)
                if file_obj:
                    # Wrap as text and read a fixed chunk
                    text_stream = io.TextIOWrapper(file_obj, encoding="utf-8", newline='')
                    start = time.time()
                    sample = text_stream.read(sample_bytes)  # read by characters (not lines)
                    elapsed = time.time() - start

                    num_lines = sample.count('\n')
                    avg_line_length = len(sample) / max(num_lines, 1)
                    estimated_total_lines = int(member.size / avg_line_length)

                    print(f"Sampled {sample_bytes} bytes in {elapsed:.2f} sec")
                    print(f"Estimated average line length: {avg_line_length:.1f} bytes")
                    print(f"Estimated total lines: {estimated_total_lines:,}")
                    return estimated_total_lines
    return 0

import pickle
from collections import defaultdict

def default_inner():
    return [0, 0.0, 0.0]

def default_outer():
    return defaultdict(default_inner)

def process_chunk(chunk):
    local_clusters = defaultdict(default_outer)

    chunk = chunk.dropna(subset=["latitude", "longitude", "taxon_id", "observed_on"])
    chunk["year"] = pd.to_datetime(chunk["observed_on"], errors="coerce").dt.year
    chunk = chunk.dropna(subset=["year"])
    chunk["year"] = chunk["year"].astype(int)
    chunk = chunk[chunk["taxon_id"].astype(int).astype(str).isin(valid_taxon_ids)]

    for _, row in chunk.iterrows():
        taxon = str(row["taxon_id"])
        year = int(row["year"])
        lat, lon = row["latitude"], row["longitude"]
        grid = get_grid_key(lat, lon)
        g = local_clusters[(taxon, year)][grid]
        g[0] += 1
        g[1] += lat
        g[2] += lon

    return local_clusters

def merge_clusters(global_clusters, local_clusters):
    for key, grid_dict in local_clusters.items():
        for grid, values in grid_dict.items():
            g = global_clusters[key][grid]
            g[0] += values[0]
            g[1] += values[1]
            g[2] += values[2]

In [19]:
import pandas as pd
import numpy as np
from scipy.interpolate import PchipInterpolator
from itertools import combinations
from geopy.distance import geodesic
from tqdm import tqdm

# Load filtered_df
filtered_df = pd.read_csv("taxon_grid_clusters_by_year.csv")

# Group to get yearly total population (sum of cluster_count)
grouped = (
    filtered_df.groupby(["taxon_id", "year"])["cluster_count"]
    .sum()
    .reset_index()
    .rename(columns={"cluster_count": "population"})
)

features = []

# Add tqdm progress bar to taxon_id loop
for taxon_id, taxon_df in tqdm(grouped.groupby("taxon_id"), desc="Processing Taxa"):
    
    taxon_df = taxon_df.sort_values("year")
    min_year = taxon_df["year"].min()
    max_year = taxon_df["year"].max()

    # Full year range including forecast up to 2024
    full_years = np.arange(min_year, 2025)
    full_df = pd.DataFrame({"year": full_years})
    full_df["taxon_id"] = taxon_id
    full_df = full_df.merge(taxon_df, on=["taxon_id", "year"], how="left")
    full_df["population"] = full_df["population"].interpolate(method="linear")

    # PCHIP extrapolation if needed
    if max_year < 2025 and len(taxon_df) >= 2:
        pchip = PchipInterpolator(
            taxon_df["year"], taxon_df["population"]
        )
        extrap_years = full_df[(full_df["year"] > max_year) & (full_df["year"] <= 2024)]["year"]
        if not extrap_years.empty:
            full_df.loc[full_df["year"].isin(extrap_years), "population"] = pchip(extrap_years)


    # Final smoothed population series
    pop_series = full_df.set_index("year")["population"]

    # Rolling window features
    pop2024 = pop_series.get(2024, np.nan)
    pop2023 = pop_series.get(2023, np.nan)
    avg2 = np.nanmean([pop2023, pop2024])

    avg5 = pop_series.loc[2020:2024].mean() if all(y in pop_series for y in range(2020, 2024)) else np.nan
    avg10 = pop_series.loc[2015:2024].mean() if all(y in pop_series for y in range(2015, 2024)) else np.nan

    # Get 2025 cluster info
    curr_clusters_df = filtered_df[
        (filtered_df["taxon_id"] == taxon_id) & (filtered_df["year"] == 2024)
    ]

    curr_clusters = curr_clusters_df.shape[0]

    coords = list(zip(curr_clusters_df["centroid_lat"], curr_clusters_df["centroid_lon"]))

    if len(coords) >= 2:
        distances = [geodesic(p1, p2).km for p1, p2 in combinations(coords, 2)]
        avg_dist_clusters = np.mean(distances)
        max_dist_clusters = np.max(distances)
    else:
        avg_dist_clusters = 0.0
        max_dist_clusters = 0.0

    features.append({
        "taxon_id": taxon_id,
        "curr_pop": pop2024,
        "avg2": avg2,
        "avg5": avg5,
        "avg10": avg10,
        "curr_clusters": curr_clusters,
        "avg_dist_clusters": avg_dist_clusters,
        "max_dist_clusters": max_dist_clusters
    })

# Combine to final DataFrame
regression_features_df = pd.DataFrame(features)

Processing Taxa:  26%|███████████████████████████████████████▌                                                                                                                   | 3759/14717 [02:11<01:43, 105.74it/s]C:\Users\backe\AppData\Local\Temp\ipykernel_25636\1374677885.py:51: RuntimeWarning: Mean of empty slice
  avg2 = np.nanmean([pop2023, pop2024])
Processing Taxa:  28%|███████████████████████████████████████████▉                                                                                                               | 4171/14717 [02:18<01:07, 155.13it/s]C:\Users\backe\AppData\Local\Temp\ipykernel_25636\1374677885.py:51: RuntimeWarning: Mean of empty slice
  avg2 = np.nanmean([pop2023, pop2024])
Processing Taxa:  43%|███████████████████████████████████████████████████████████████████▎                                                                                       | 6391/14717 [02:51<01:19, 104.49it/s]C:\Users\backe\AppData\Local\Temp\ipykernel_25636\1374677885.py:51:

Prepare Regression Data for Conservation Status Predictions

In [29]:
#print(regression_features_df)
conservation_df = pd.read_csv('backend/data/filtered_species_with_conservation_status.csv')
new_df = regression_features_df.merge(conservation_df, on='taxon_id', how='left')
print(new_df.columns)
new_df = new_df[[
    'curr_pop','avg2', 'avg5', 'avg10', 'curr_clusters',
    'avg_dist_clusters', 'max_dist_clusters',
    'ancestry', 'conservation_status'
]]

#if conservation status == 'threatened', conservation_status=1, else 0
new_df["conservation_status"] = (new_df["conservation_status"] == "threatened").astype(int)

#remove some NaNs
new_df["curr_pop"] = new_df["curr_pop"].fillna(0)
new_df["avg2"] = new_df["avg2"].fillna(new_df["curr_pop"])
new_df["avg5"] = new_df["avg5"].fillna(new_df["avg2"])
new_df["avg10"] = new_df["avg10"].fillna(new_df["avg5"])

#reduce ancestry to only the 2 groups
target_classes = ["40151", "3"]

def extract_target_class(ancestry):
    parts = ancestry.split('/')
    for tid in target_classes:
        if tid in parts:
            return 1 if tid == "3" else 0
    return ancestry  # Or return "other" / None if you want to drop non-targets

# Apply transformation
new_df["ancestry"] = new_df["ancestry"].astype(str).apply(extract_target_class)

from sklearn.preprocessing import StandardScaler

# Select numeric columns to normalize
num_cols = ['curr_pop','avg2', 'avg5', 'avg10', 'curr_clusters', 'avg_dist_clusters', 'max_dist_clusters']
scaler = StandardScaler()
new_df[num_cols] = scaler.fit_transform(new_df[num_cols])

print(new_df)

Index(['taxon_id', 'curr_pop', 'avg2', 'avg5', 'avg10', 'curr_clusters',
       'avg_dist_clusters', 'max_dist_clusters', 'ancestry', 'rank_level',
       'rank', 'name', 'active', 'conservation_status'],
      dtype='object')
       curr_pop      avg2      avg5     avg10  curr_clusters  \
0      0.992508  1.259202  1.146376  1.245539       2.233065   
1     -0.002665 -0.003246 -0.025871 -0.020163       0.834223   
2     -0.167331 -0.161052 -0.158858 -0.160677      -0.253766   
3     -0.070638 -0.074647 -0.088791 -0.077140       1.611357   
4     -0.153928 -0.157689 -0.153880 -0.153181      -0.098339   
...         ...       ...       ...       ...            ...   
14712 -0.173075 -0.173987 -0.167919 -0.171173      -0.486907   
14713 -0.169964 -0.171271 -0.165558 -0.167585      -0.486907   
14714 -0.171160 -0.172694 -0.165622 -0.167318      -0.409193   
14715 -0.172596 -0.173211 -0.166771 -0.168389      -0.409193   
14716 -0.173075 -0.173987 -0.167919 -0.171173      -0.409193   

    

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Features and labels
reg_features = ['ancestry', 'curr_pop', 'avg2', 'avg5', 'avg10',
                'curr_clusters', 'avg_dist_clusters', 'max_dist_clusters']
X_reg = new_df[reg_features]
y = new_df['conservation_status']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_reg, y, test_size=0.2, random_state=42)

from xgboost import XGBClassifier

# Define models
models = {
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=1000),
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "Support Vector Machine": SVC(class_weight='balanced', probability=True, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(class_weight='balanced', random_state=42),
    "XGBoost" : XGBClassifier(scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(), use_label_encoder=False, eval_metric="logloss", random_state=42)
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"=== {name} ===")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))


=== Logistic Regression ===
Accuracy: 0.6222826086956522
              precision    recall  f1-score   support

           0       0.79      0.69      0.74      2273
           1       0.27      0.39      0.32       671

    accuracy                           0.62      2944
   macro avg       0.53      0.54      0.53      2944
weighted avg       0.67      0.62      0.64      2944

=== Naive Bayes ===
Accuracy: 0.27683423913043476
              precision    recall  f1-score   support

           0       0.90      0.07      0.13      2273
           1       0.24      0.97      0.38       671

    accuracy                           0.28      2944
   macro avg       0.57      0.52      0.26      2944
weighted avg       0.75      0.28      0.19      2944

=== Random Forest ===
Accuracy: 0.7102581521739131
              precision    recall  f1-score   support

           0       0.81      0.82      0.81      2273
           1       0.36      0.35      0.36       671

    accuracy            

C:\Users\backe\Documents\GitHub\species_tracker\.venv\lib\site-packages\xgboost\training.py:183: UserWarning: [22:39:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


We have determined SVM is the best

In [36]:
import numpy as np
from sklearn.metrics import precision_recall_curve, f1_score

# Get predicted probabilities for class 1
svm_probs = models["Support Vector Machine"].predict_proba(X_test)[:, 1]

# Try different thresholds
thresholds = np.linspace(0.1, 0.9, 50)
f1_scores = []
best_thresh = 0.5
best_f1 = 0

for thresh in thresholds:
    y_pred_thresh = (svm_probs >= thresh).astype(int)
    score = f1_score(y_test, y_pred_thresh)
    f1_scores.append(score)
    if score > best_f1:
        best_f1 = score
        best_thresh = thresh

print(f"Best threshold for SVM: {best_thresh:.2f} | F1-score: {best_f1:.4f}")


Best threshold for SVM: 0.28 | F1-score: 0.4662


In [ ]:
probs = models["Support Vector Machine"].predict_proba(X_test)
y_pred = (probs[:, 1] >= 0.28).astype(int)        # your custom threshold
confidences = probs.max(axis=1)                 # confidence = max probability
decision_scores = models["Support Vector Machine"].decision_function(X_test)

results_df = pd.DataFrame({
    "true_label": y_test,
    "predicted": y_pred,
    "confidence": confidences
})

#print(results_df)

       true_label  predicted  confidence
10275           1          1    0.619120
10643           0          1    0.651268
9490            0          1    0.651603
13025           0          0    0.802738
11459           1          1    0.651488
...           ...        ...         ...
6598            1          1    0.651560
14632           0          1    0.651268
13104           0          1    0.651805
3844            0          0    0.885960
14283           0          1    0.651920

[2944 rows x 3 columns]


In [43]:
import joblib

# Bundle everything you need
print(reg_features)
bundle = {
    "model": models["Support Vector Machine"],
    "scaler": scaler,
    "threshold": 0.28,
    "features": reg_features  # optional but useful
}

# Save it as one .joblib file
joblib.dump(bundle, "svm_model_bundle.joblib")


['ancestry', 'curr_pop', 'avg2', 'avg5', 'avg10', 'curr_clusters', 'avg_dist_clusters', 'max_dist_clusters']


['svm_model_bundle.joblib']